In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats

In [2]:
rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')
rating_data.columns = ['UserID', 'ItemID', 'Rating', 'Timestamp']
rating_data.drop(columns=['Timestamp'], axis=1, inplace=True)

<ipython-input-2-2e5bf9d6dc83>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')


In [3]:
rating_data

,UserID,ItemID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [4]:
item_cnt = max(rating_data['ItemID']) + 1
users_cnt = max(rating_data['UserID']) + 1
item_cnt, users_cnt

(3953, 6041)

In [5]:
rating_train = rating_data.sample(frac=0.8, random_state=200)
rating_test = rating_data.drop(rating_train.index)